# TP d'Introduction à l'Économétrie
## Modèle Linéaire Simple

Ce TP reprend les concepts fondamentaux du modèle linéaire simple présentés dans le cours.

Nous allons explorer les méthodes d'estimation (MCO et maximum de vraisemblance), les hypothèses du modèle, les tests statistiques et l'interprétation des résultats.

## 1. Importation des bibliothèques nécessaires

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats
import statsmodels.api as sm
from statsmodels.graphics.gofplots import ProbPlot

# Configuration pour de meilleurs graphiques
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 14
np.random.seed(0)  # Pour reproduire les résultats

## 2. Génération de données simulées

Nous allons simuler un modèle linéaire simple de la forme : $Y_i = \alpha + \beta X_i + u_i$

Où :
- $\alpha$ est la constante (l'ordonnée à l'origine)
- $\beta$ est le coefficient de la variable explicative
- $u_i$ est le terme d'erreur qui suit une loi normale

In [ ]:
# Paramètres du modèle
alpha_vrai = 10  # Vraie valeur de alpha
beta_vrai = 2    # Vraie valeur de beta
sigma_u = 3      # Écart-type du terme d'erreur
n = 100          # Nombre d'observations

# Génération de la variable explicative X
X = np.random.uniform(0, 10, n)

# Génération du terme d'erreur u (suivant une loi normale centrée de variance sigma_u²)
u = np.random.normal(0, sigma_u, n)

# Génération de la variable dépendante Y selon le modèle
Y = alpha_vrai + beta_vrai * X + u

# Création d'un DataFrame pour stocker les données
df = pd.DataFrame({
    'X': X,
    'Y': Y,
    'u': u  # On garde les erreurs pour vérification ultérieure
})

# Affichage des premières lignes du DataFrame
df.head()

## 3. Visualisation des données

In [ ]:
plt.figure(figsize=(10, 6))
plt.scatter(X, Y, alpha=0.7)
plt.plot(X, alpha_vrai + beta_vrai * X, 'r-', label='Vraie relation : Y = {} + {}X'.format(alpha_vrai, beta_vrai))
plt.xlabel('X')
plt.ylabel('Y')
plt.title('Relation entre X et Y avec le vrai modèle')
plt.legend()
plt.grid(True)
plt.show()

## 4. Estimation par la méthode des Moindres Carrés Ordinaires (MCO)

Rappel des formules des estimateurs MCO :

$$\hat{\beta} = \frac{\sum_{i=1}^{n}(X_i - \bar{X})(Y_i - \bar{Y})}{\sum_{i=1}^{n}(X_i - \bar{X})^2}$$

$$\hat{\alpha} = \bar{Y} - \hat{\beta}\bar{X}$$

Nous allons implémenter ces formules manuellement puis comparer avec l'estimateur de statsmodels.

In [ ]:
# Calcul manuel des estimateurs MCO
X_mean = np.mean(X)
Y_mean = np.mean(Y)

# Calcul de beta
numerator = np.sum((X - X_mean) * (Y - Y_mean))
denominator = np.sum((X - X_mean)**2)
beta_hat = numerator / denominator

# Calcul de alpha
alpha_hat = Y_mean - beta_hat * X_mean

print(f"Estimateur MCO manuel pour α: {alpha_hat:.4f}")
print(f"Estimateur MCO manuel pour β: {beta_hat:.4f}")
print(f"Valeurs réelles: α = {alpha_vrai}, β = {beta_vrai}")

In [ ]:
# Utilisation de statsmodels pour l'estimation MCO
X_sm = sm.add_constant(X)  # Ajoute une colonne de 1 pour la constante
model = sm.OLS(Y, X_sm)    # Ordinary Least Squares
results = model.fit()

# Affichage du résumé des résultats
print(results.summary())

## 5. Calcul et interprétation des résidus

In [ ]:
# Calcul des valeurs prédites
Y_pred = alpha_hat + beta_hat * X

# Calcul des résidus
residus = Y - Y_pred

# Ajout des résidus et valeurs prédites au DataFrame
df['Y_pred'] = Y_pred
df['residus'] = residus

# Estimation de la variance des erreurs
sigma_u_hat = np.sqrt(np.sum(residus**2) / (n - 2))

print(f"Estimation de l'écart-type des erreurs: {sigma_u_hat:.4f}")
print(f"Vraie valeur de l'écart-type des erreurs: {sigma_u:.4f}")

In [ ]:
# Visualisation des résidus
plt.figure(figsize=(16, 6))

# Premier graphique: résidus en fonction de X
plt.subplot(1, 2, 1)
plt.scatter(X, residus, alpha=0.7)
plt.axhline(y=0, color='r', linestyle='-')
plt.xlabel('X')
plt.ylabel('Résidus')
plt.title('Résidus en fonction de X')
plt.grid(True)

# Deuxième graphique: histogramme des résidus
plt.subplot(1, 2, 2)
sns.histplot(residus, kde=True)
plt.xlabel('Résidus')
plt.title('Distribution des résidus')

plt.tight_layout()
plt.show()

## 6. Vérification des hypothèses du modèle linéaire simple

Rappel des hypothèses :
1. La distribution de l'erreur u est indépendante de X
2. L'erreur est centrée et de variance constante (homoscédasticité)
3. Les coefficients α et β sont constants
4. Les résidus suivent une loi normale
5. Non-autocorrélation des résidus

In [ ]:
# Diagnostic graphique des résidus
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# Graphique 1: Résidus vs Valeurs prédites
axes[0, 0].scatter(Y_pred, residus, alpha=0.7)
axes[0, 0].axhline(y=0, color='r', linestyle='-')
axes[0, 0].set_xlabel('Valeurs prédites')
axes[0, 0].set_ylabel('Résidus')
axes[0, 0].set_title('Résidus vs Valeurs prédites (homoscédasticité)')
axes[0, 0].grid(True)

# Graphique 2: QQ-plot pour tester la normalité
qq_plot = ProbPlot(residus)
qq_plot.qqplot(line='s', ax=axes[0, 1])
axes[0, 1].set_title('QQ-Plot des résidus (normalité)')

# Graphique 3: Résidus vs ordre (autocorrélation)
axes[1, 0].plot(range(len(residus)), residus, marker='o', linestyle='none', alpha=0.7)
axes[1, 0].axhline(y=0, color='r', linestyle='-')
axes[1, 0].set_xlabel('Ordre des observations')
axes[1, 0].set_ylabel('Résidus')
axes[1, 0].set_title('Résidus vs Ordre (autocorrélation)')
axes[1, 0].grid(True)

# Graphique 4: Distribution des résidus standardisés
standardized_residuals = residus / np.std(residus)
sns.histplot(standardized_residuals, kde=True, ax=axes[1, 1])
axes[1, 1].set_xlabel('Résidus standardisés')
axes[1, 1].set_title('Distribution des résidus standardisés')

plt.tight_layout()
plt.show()

In [ ]:
# Test de normalité des résidus (Shapiro-Wilk)
shapiro_test = stats.shapiro(residus)
print(f"Test de Shapiro-Wilk pour la normalité des résidus:")
print(f"Statistique W: {shapiro_test[0]:.4f}")
print(f"p-value: {shapiro_test[1]:.4f}")
print(f"Conclusion: {'Résidus normaux' if shapiro_test[1] > 0.05 else 'Résidus non normaux'}")

# Test d'homoscédasticité (Breusch-Pagan)
bp_test = sm.stats.diagnostic.het_breuschpagan(residus, results.model.exog)
print("\nTest de Breusch-Pagan pour l'homoscédasticité:")
print(f"Statistique LM: {bp_test[0]:.4f}")
print(f"p-value: {bp_test[1]:.4f}")
print(f"Conclusion: {'Homoscédasticité' if bp_test[1] > 0.05 else 'Hétéroscédasticité'}")

## 7. Analyse de la variance et qualité de l'ajustement (R²)

Rappel de la décomposition de la variance totale :

$$\sum_{i=1}^{n}(Y_i - \bar{Y})^2 = \sum_{i=1}^{n}(\hat{Y}_i - \bar{Y})^2 + \sum_{i=1}^{n}(Y_i - \hat{Y}_i)^2$$

$$SCT = SCE + SCR$$

Et le coefficient de détermination :

$$R^2 = \frac{SCE}{SCT} = 1 - \frac{SCR}{SCT}$$

In [ ]:
# Calcul de la somme des carrés totale (SCT)
SCT = np.sum((Y - Y_mean)**2)

# Calcul de la somme des carrés expliquée (SCE)
SCE = np.sum((Y_pred - Y_mean)**2)

# Calcul de la somme des carrés résiduelle (SCR)
SCR = np.sum((Y - Y_pred)**2)

# Calcul du R²
R2 = SCE / SCT
# Vérification: R² = 1 - SCR/SCT
R2_verif = 1 - SCR / SCT

print(f"Somme des carrés totale (SCT): {SCT:.4f}")
print(f"Somme des carrés expliquée (SCE): {SCE:.4f}")
print(f"Somme des carrés résiduelle (SCR): {SCR:.4f}")
print(f"Vérification: SCT = SCE + SCR -> {SCT:.4f} = {SCE:.4f} + {SCR:.4f} -> {SCE + SCR:.4f}")
print(f"\nR² calculé manuellement: {R2:.4f}")
print(f"R² calculé par vérification: {R2_verif:.4f}")
print(f"R² fourni par statsmodels: {results.rsquared:.4f}")

In [ ]:
# Visualisation de la décomposition de la variance
plt.figure(figsize=(12, 8))
plt.scatter(X, Y, alpha=0.7, label='Observations')
plt.plot(X, Y_pred, 'r-', linewidth=2, label=f'Régression: Y = {alpha_hat:.4f} + {beta_hat:.4f}X')
plt.plot([X_mean, X_mean], [0, Y_mean], 'k--', alpha=0.5)
plt.axhline(y=Y_mean, color='g', linestyle='--', alpha=0.5, label=f'Moyenne de Y: {Y_mean:.4f}')

# Ajouter quelques lignes pour illustrer la décomposition de la variance
for i in range(0, n, 10):  # Afficher seulement quelques lignes pour clarté
    plt.plot([X[i], X[i]], [Y[i], Y_pred[i]], 'b-', alpha=0.3)  # SCR
    plt.plot([X[i], X[i]], [Y_pred[i], Y_mean], 'g-', alpha=0.3)  # SCE

plt.xlabel('X')
plt.ylabel('Y')
plt.title(f'Régression linéaire avec décomposition de la variance (R² = {R2:.4f})')
plt.legend()
plt.grid(True)
plt.show()

## 8. Tests statistiques sur les coefficients

Nous allons tester les hypothèses : $H_0: \beta = 0$ vs $H_1: \beta \neq 0$

Rappel : sous les hypothèses du modèle linéaire simple :

$$\frac{\hat{\beta} - \beta}{\sqrt{s^2 C_\beta}} \sim T(n-2)$$

Où $C_\beta = \frac{1}{n\sum_{i=1}^{n}(X_i - \bar{X})^2}$

In [ ]:
# Calcul de l'écart-type de beta
C_beta = 1 / (n * np.sum((X - X_mean)**2))
se_beta = np.sqrt(sigma_u_hat**2 * C_beta)

# Statistique de test pour H0: beta = 0
t_stat = beta_hat / se_beta

# Valeur critique et p-value
dof = n - 2  # degrés de liberté
p_value = 2 * (1 - stats.t.cdf(abs(t_stat), dof))  # test bilatéral

# Intervalle de confiance à 95%
t_crit = stats.t.ppf(0.975, dof)  # quantile à 97.5% (test bilatéral à 5%)
ci_lower = beta_hat - t_crit * se_beta
ci_upper = beta_hat + t_crit * se_beta

print(f"Test de significativité de β:")
print(f"H0: β = 0 vs H1: β ≠ 0")
print(f"Estimateur de β: {beta_hat:.4f}")
print(f"Écart-type de β: {se_beta:.4f}")
print(f"Statistique t: {t_stat:.4f}")
print(f"p-value: {p_value:.6f}")
print(f"Conclusion: {'Rejet de H0' if p_value < 0.05 else 'Non-rejet de H0'} au seuil de 5%")
print(f"\nIntervalle de confiance à 95% pour β: [{ci_lower:.4f}, {ci_upper:.4f}]")
print(f"La vraie valeur de β ({beta_vrai}) est-elle dans l'intervalle? {'Oui' if ci_lower <= beta_vrai <= ci_upper else 'Non'}")

## 9. Prévisions avec le modèle estimé

Maintenant que nous avons estimé notre modèle, nous pouvons l'utiliser pour faire des prévisions.

In [ ]:
# Nouvelle valeur de X pour la prévision
X_new = 15  # Une valeur en dehors de la plage des données

# Prévision ponctuelle
Y_pred_new = alpha_hat + beta_hat * X_new

# Intervalle de prévision à 95%
# Formule: Y_pred ± t_{n-2, 0.975} * s * sqrt(1 + 1/n + (X_new - X̄)²/Σ(X_i - X̄)²)
prediction_variance = sigma_u_hat**2 * (1 + 1/n + ((X_new - X_mean)**2) / np.sum((X - X_mean)**2))
prediction_se = np.sqrt(prediction_variance)
prediction_margin = t_crit * prediction_se

pred_lower = Y_pred_new - prediction_margin
pred_upper = Y_pred_new + prediction_margin

print(f"Prévision pour X = {X_new}:")
print(f"Valeur prédite: {Y_pred_new:.4f}")
print(f"Intervalle de prévision à 95%: [{pred_lower:.4f}, {pred_upper:.4f}]")

# Visualisation
X_range = np.linspace(0, 16, 100)
Y_range_pred = alpha_hat + beta_hat * X_range

plt.figure(figsize=(12, 8))
plt.scatter(X, Y, alpha=0.7, label='Observations')
plt.plot(X_range, Y_range_pred, 'r-', linewidth=2, label=f'Régression: Y = {alpha_hat:.4f} + {beta_hat:.4f}X')
plt.scatter(X_new, Y_pred_new, color='green', s=100, marker='X', label=f'Prévision pour X = {X_new}')

# Ajout de l'intervalle de prévision
plt.fill_between(X_range, 
                 alpha_hat + beta_hat * X_range - t_crit * sigma_u_hat * np.sqrt(1 + 1/n + ((X_range - X_mean)**2) / np.sum((X - X_mean)**2)),
                 alpha_hat + beta_hat * X_range + t_crit * sigma_u_hat * np.sqrt(1 + 1/n + ((X_range - X_mean)**2) / np.sum((X - X_mean)**2)),
                 color='gray', alpha=0.2, label='Intervalle de prévision à 95%')

plt.vlines(X_new, pred_lower, pred_upper, color='green', linestyle='--', alpha=0.7)
plt.xlabel('X')
plt.ylabel('Y')
plt.title('Régression linéaire avec prévision')
plt.legend()
plt.grid(True)
plt.show()

## 10. Différents types de modèles et interprétation des coefficients

Comme indiqué dans le cours, il existe différentes formes de modèles linéaires selon la transformation appliquée aux variables. Nous allons explorer quatre types de modèles :

1. lin-lin: $Y = \alpha + \beta X + u$
2. log-lin: $\log(Y) = \alpha + \beta X + u$
3. lin-log: $Y = \alpha + \beta \log(X) + u$
4. log-log: $\log(Y) = \alpha + \beta \log(X) + u$

In [ ]:
# Création de nouvelles variables avec des distributions positives
np.random.seed(0)
X_pos = np.random.lognormal(0, 0.5, n)  # Distribution log-normale pour assurer X > 0
beta_log_lin = 0.05  # Pour log-lin
beta_lin_log = 5     # Pour lin-log
beta_log_log = 0.8   # Pour log-log

# Génération des Y pour les différents modèles
Y_lin_lin = 10 + 2 * X_pos + np.random.normal(0, 1, n)
Y_log_lin = np.exp(2 + beta_log_lin * X_pos + np.random.normal(0, 0.1, n))
Y_lin_log = 5 + beta_lin_log * np.log(X_pos) + np.random.normal(0, 1, n)
Y_log_log = np.exp(1 + beta_log_log * np.log(X_pos) + np.random.normal(0, 0.1, n))

# Création d'un DataFrame pour ces données
df_models = pd.DataFrame({
    'X': X_pos,
    'Y_lin_lin': Y_lin_lin,
    'Y_log_lin': Y_log_lin,
    'Y_lin_log': Y_lin_log,
    'Y_log_log': Y_log_log,
    'log_X': np.log(X_pos),
    'log_Y_log_lin': np.log(Y_log_lin),
    'log_Y_log_log': np.log(Y_log_log)
})

# Affichage des premières lignes du DataFrame
df_models.head()